In [1]:
!pip install scikit-learn
import sklearn
sklearn.__version__

'1.8.0'

# Classificação Automática — Sofascore  
## Premier League 2023/2024

**Grupo:**  
26535 Pedro Simões  
26025 Carlos Brandão  
26027 Pedro Carneiro


## 1. Objetivo de Negócio

O objetivo deste projeto é aplicar técnicas de classificação automática para prever se um jogador da **Premier League 2023/2024** apresenta uma **classificação elevada no Sofascore**, com base nas suas estatísticas individuais de desempenho.

Este tipo de análise pode apoiar o **scouting**, a **avaliação de jogadores** e a **análise de desempenho**, ao permitir identificar atletas com elevado impacto competitivo.


## 2. Descrição do Dataset

O dataset contém estatísticas individuais dos jogadores da **Premier League 2023/2024**, recolhidas a partir do Sofascore.

### Variáveis Disponíveis:
- Nome do Jogador
- Classificação (Rating)
- Dribles Bem-sucedidos
- Dribles Bem-sucedidos (%)
- Grandes Oportunidades Falhadas
- Grandes Oportunidades Criadas
- Posse de Bola Perdida
- Nome da Equipa
- Liga
- Época

In [2]:
import pandas as pd

df = pd.read_csv("premier_league_23_24.csv")

df.head()


,Player Name,Rating,Successful Dribbles,Successful Dribbles (%),Big Chances Missed,Big Chances Created,Possession Lost,Team Name,League,Season
0,Rodri,8.01,42,75.00,2,8,361,Manchester City,Premier League,2023-2024
1,Kevin De Bruyne,7.93,10,32.26,1,21,296,Manchester City,Premier League,2023-2024
2,Phil Foden,7.79,47,50.54,7,10,417,Manchester City,Premier League,2023-2024
3,Bruno Fernandes,7.75,19,47.50,7,21,695,Manchester United,Premier League,2023-2024
4,Michael Olise,7.74,40,54.79,3,9,299,Crystal Palace,Premier League,2023-2024


## 3. Critérios de Seleção dos Dados

Para a tarefa de classificação, foram selecionadas apenas as variáveis relacionadas com o desempenho individual dos jogadores.

### Atributos Selecionados
- Dribles Bem-sucedidos
- Dribles Bem-sucedidos (%)
- Grandes Oportunidades Criadas
- Grandes Oportunidades Falhadas
- Posse de Bola Perdida

### Variáveis Excluídas
- Nome do Jogador (identificador)
- Classificação (Rating) — utilizada apenas para criar a variável alvo
- Liga e Época (valores constantes)

## 4. Definição da Variável Alvo

Os jogadores com uma classificação no **top 20%** da distribuição são considerados jogadores de alto desempenho.

Foi criada uma variável alvo binária `High_Rating`:
- 1 → Classificação elevada
- 0 → Caso contrário


In [3]:
# Create target variable
threshold = df["Rating"].quantile(0.80)
df["High_Rating"] = (df["Rating"] >= threshold).astype(int)

df["High_Rating"].value_counts()


High_Rating
0    303
1     84
Name: count, dtype: int64

## 5. Preparação dos Dados

In [6]:
# Select features and target
features = [
    "Successful Dribbles",
    "Successful Dribbles (%)",
    "Big Chances Created",
    "Big Chances Missed",
    "Possession Lost"
]

X = df[features]
y = df["High_Rating"]

# Handle missing values
X = X.fillna(X.median())


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## 6. Algoritmos de Classificação

Foram aplicados três algoritmos de classificação:
- Regressão Logística
- Árvore de Decisão
- Random Forest


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [9]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    print(f"\n{name}")
    print(classification_report(y_test, preds))



Logistic Regression
              precision    recall  f1-score   support

           0       0.77      0.98      0.86        54
           1       0.89      0.33      0.48        24

    accuracy                           0.78        78
   macro avg       0.83      0.66      0.67        78
weighted avg       0.81      0.78      0.75        78


Decision Tree
              precision    recall  f1-score   support

           0       0.81      0.93      0.86        54
           1       0.75      0.50      0.60        24

    accuracy                           0.79        78
   macro avg       0.78      0.71      0.73        78
weighted avg       0.79      0.79      0.78        78


Random Forest
              precision    recall  f1-score   support

           0       0.76      0.93      0.83        54
           1       0.67      0.33      0.44        24

    accuracy                           0.74        78
   macro avg       0.71      0.63      0.64        78
weighted avg       0.73

## 7. Otimização do Modelo

O modelo com o melhor desempenho inicial foi otimizado através de ajuste de hiperparâmetros (hyperparameter tuning).


In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}

grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring="f1"
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_


In [12]:
# Evaluation of optimized model
preds = best_model.predict(X_test)

print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))


              precision    recall  f1-score   support

           0       0.76      0.94      0.84        54
           1       0.73      0.33      0.46        24

    accuracy                           0.76        78
   macro avg       0.74      0.64      0.65        78
weighted avg       0.75      0.76      0.72        78

[[51  3]
 [16  8]]


## 8. Resultados e Conclusões

Os resultados indicam que é possível prever jogadores com **classificação elevada** na **Premier League 2023/2024** utilizando métricas de desempenho individual relacionadas com a **eficiência no drible**, a **criação de oportunidades** e a **gestão da posse de bola**.

O modelo **Random Forest otimizado** apresentou o melhor desempenho global. No entanto, a análise encontra-se limitada pelo **número reduzido de variáveis** e pela **ausência de métricas defensivas**.

Como trabalho futuro, poderão ser incluídas **outras ligas**, **diferentes épocas** ou **indicadores de desempenho mais detalhados**, de forma a melhorar a capacidade preditiva do modelo e a sua aplicabilidade prática.